<a href="https://colab.research.google.com/github/2kunal6/UniBonn/blob/master/cuda_lab/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from torchvision import datasets, transforms
import numpy as np
from tqdm import tqdm

train_mnist = datasets.MNIST("", train=True, transform=transforms.ToTensor(), download=True)

test_mnist = datasets.MNIST("", train=False, transform=transforms.ToTensor(), download=True)

train = []
test = []
for data in train_mnist:
  train.append([data[0].view(28*28).numpy(), np.eye(10)[data[1]]])

for data in test_mnist:
  test.append([data[0].view(28*28).numpy(), np.eye(10)[data[1]]])


print(len(train))
print(len(test))
print(len(train[0][0]), type(train[0][0]))

60000
10000
784 <class 'numpy.ndarray'>


In [14]:
np.random.seed(1)
num_features = 784
num_classes = 10
weights = np.random.random((num_features, 1))
bias = 0.0

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def forward(x):
  return np.dot(x, weights)

# MSE
def calc_loss(y, y_pred):
  #print(y)
  #print(y_pred)
  return ((y-y_pred)**2).mean()

# gradient of MSE dJ/dw = 1/N 2x (w*x - y)
def calc_gradient(x, y, y_pred):
  return (np.dot(2*x, y_pred - y)).mean()

#print("prediction before training : ", forward(10))

learning_rate = 0.01

for epoch in range(15):
  BATCH_SIZE = 20
  for i in tqdm(range(0, len(train), BATCH_SIZE)): 
    batch = train[i:i+BATCH_SIZE+1]

    batch_X = [i[0] for i in batch]
    batch_y = [i[1] for i in batch]

    y_pred = forward(batch_X)

    print(y_pred)
    break
    
    squared_error = (y_pred - batch_y)**2
    print(squared_error)

    mse_loss = 1.0/(2*BATCH_SIZE) * (squared_error.sum())
    loss = calc_loss(y, y_pred)

    grad = calc_gradient(x, y, y_pred)
    break

  w -= learning_rate * grad

  print("epoch : ", epoch, "weight : ", w, "loss : ", loss)


  0%|          | 0/3000 [00:00<?, ?it/s]

[[56.23529669]
 [64.05402136]
 [40.62964516]
 [30.03184308]
 [46.03123963]
 [54.90223055]
 [35.40564861]
 [70.73066712]
 [21.87902984]
 [45.47700791]
 [53.57369739]
 [25.99813755]
 [71.29293838]
 [55.6469619 ]
 [22.34394569]
 [51.63541635]
 [45.1141025 ]
 [53.57636123]
 [26.40070012]
 [37.33223489]
 [69.1202414 ]]


NameError: ignored